In [1]:
import pandas as pd
import numpy as np
import sklearn
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from acquire import new_property_data
from acquire import get_zillow_data

In [2]:
# acquiring fresh set of data without alterations
property_data = get_zillow_data()

In [3]:
# specifying columns we'll need
property_data = property_data[['regionidzip', 'regionidcounty', 'taxamount', 'taxvaluedollarcnt']]

# renaming columns for readability and understanding
property_data = property_data.rename(columns = {'regionidzip' : 'zipcode', 'regionidcounty' : 'county_id', "taxamount" : "tax_amount", "taxvaluedollarcnt" : "tax_dollar_value" })

# previewing df
property_data.head()

,zipcode,county_id,tax_amount,tax_dollar_value
0,97318.0,3101.0,2319.90,136104.0
1,96018.0,3101.0,543.69,35606.0
2,96037.0,3101.0,7673.19,614000.0
3,96023.0,3101.0,3267.47,274237.0
4,96008.0,3101.0,2926.19,245906.0


In [4]:
# reading data from external csv with zip, state, and county name info
county_state_zip = pd.read_csv('county.csv')

# renaming column for readability
county_state_zip = county_state_zip.rename(columns = {'countyname' : 'county_name'})

# previewing data
county_state_zip.head()

,Unnamed: 0,county_name,state,zipcode
0,0,Siskiyou County,CA,96023.0
1,444,Shasta County,CA,96008.0
2,702,Shasta County,CA,96047.0
3,1128,Trinity County,CA,96024.0
4,1458,Lassen County,CA,96117.0


In [6]:
# merging DFs by matching zipcodes
property_locations = pd.merge(county_state_zip, property_data, on = 'zipcode')

# creating individual property tax rate column 
property_locations['property_tax_rate'] = property_locations.tax_amount / property_locations.tax_dollar_value

# dropping extra column
property_locations = property_locations.drop(columns=['Unnamed: 0'])

# previewing data frame
property_locations.head()

,county_name,state,zipcode,county_id,tax_amount,tax_dollar_value,property_tax_rate
0,Siskiyou County,CA,96023.0,3101.0,3267.47,274237.0,0.011915
1,Siskiyou County,CA,96023.0,3101.0,5836.52,480000.0,0.012159
2,Siskiyou County,CA,96023.0,3101.0,6992.82,577270.0,0.012114
3,Siskiyou County,CA,96023.0,3101.0,3826.16,318089.0,0.012029
4,Siskiyou County,CA,96023.0,3101.0,9861.47,815862.0,0.012087


In [8]:
# grouping by county and finding average tax rate amoung properties there, ie. county tax rate
county_tax = pd.DataFrame(property_locations.groupby('county_name').property_tax_rate.mean())

# renaming column because it now holds the county tax rate, not individual property rates
county_tax = county_tax.rename(columns = {'property_tax_rate' : 'county_tax_rate'})

# displaying df
county_tax

,county_tax_rate
county_name,
Alpine County,0.012234
Benton County,0.019118
Colusa County,0.012810
El Dorado County,0.012188
Glenn County,0.014196
Lane County,0.013898
Lassen County,0.012027
Lincoln County,0.013898
Marion County,0.019089
